# Collect UDP Throughput as Metric
## to Laptop server (s) from the rest RPIs client (c)

**IP table:**

| Laptop    | RPI 0      | RPI 1      | RPI 2     | RPI 3     |
|:---------:|:----------:|:----------:|:---------:|:---------:|
| 10.42.0.2 | 10.42.0.14 | 10.42.0.12 | 10.42.0.7 | 10.42.0.3 |

In [ ]:
import sys

sys.path.append('methods')
sys.path.append('figures')
# !pip install -e ../package/


from methods.route import iperf3_udp_throughput, Iperf3Out
import json
import matplotlib.pyplot as plt
import numpy as np
from pandas import DataFrame as DF
import time
from copy import deepcopy

In [ ]:
time.sleep(10)
# udp vars names sets by ip host
# Disclaimer: its more easy for coder

# Laptop (s) -- RPI 3 (c)
udp3 = iperf3_udp_throughput('10.42.0.3','10.42.0.2',6262,24,40,R=True)
udp3 = Iperf3Out(**udp3)

time.sleep(5)
# Laptop (s) -- RPI 0 (c)
udp14 = iperf3_udp_throughput('10.42.0.14','10.42.0.2',6262,24,40,R=True)
udp14 = Iperf3Out(**udp14)

time.sleep(5)
# Laptop (s) -- RPI 1 (c)
udp12 = iperf3_udp_throughput('10.42.0.12','10.42.0.2',6262,24,40,R=True)
udp12 = Iperf3Out(**udp12)

time.sleep(5)
# Laptop (s) -- RPI 2 (c)
udp7 = iperf3_udp_throughput('10.42.0.7','10.42.0.2',6262,24,40,R=True)
udp7 = Iperf3Out(**udp7)


In [ ]:
udp3Mbps = [inter['sum']['bits_per_second']/(10**6) for inter in udp3.intervals]
udp12Mbps = [inter['sum']['bits_per_second']/(10**6) for inter in udp12.intervals]
udp7Mbps = [inter['sum']['bits_per_second']/(10**6) for inter in udp7.intervals]
udp14Mbps = [inter['sum']['bits_per_second']/(10**6) for inter in udp14.intervals]

In [ ]:
udpdf = DF(
    np.array([udp3Mbps,udp7Mbps,udp12Mbps,udp14Mbps]).T
    ,columns=[
        'Laptop (s) -- RPI 3 (c)',
        'Laptop (s) -- RPI 2 (c)',
        'Laptop (s) -- RPI 1 (c)',
        'Laptop (s) -- RPI 0 (c)'])

Think something this is hard coded for vars

In [ ]:
# here we set room id (0 up to 2) and action (moving status moving/standing)
room = "0"
action = "moving"
dict_base = {
    "attribute":{
        "room": room,
        "action": action
    },
    "udp": udpdf.to_dict()
}

In [ ]:
plt.figure(figsize=(16,8))
markers = ['*', 'o', '^', 's']
for marker, (k, v) in zip(markers, dict_base.items()):
    udp = [vv for _, vv in v.items()]
    plt.plot(udp,'{}-'.format(marker),label=k, markersize=12)

plt.legend(fontsize=20)
plt.xlabel(r'$time \, (s)$', fontsize=20)
plt.ylabel(r"$ UDP \, Throughput \, (Mbps)$", fontsize=20)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)

save_obj = ['udp-thr-metric']
if save_obj is not None:
    plt.savefig('DataFigures/2ornn/{}.eps'.format(*save_obj), format='eps')

In [ ]:
dataset_path = "../data/dataset"
examples = "test_data"
write = False
examples_path = '{}/{}.json'.format(dataset_path, examples)
try:
    with open(examples_path, 'r') as json_file:
        raw_data=json_file.read()
    objs= json.loads(raw_data)
except FileNotFoundError:
    print("Not find path: {}".format(examples_path))
    objs = deepcopy(dict_base)

last_example_idx = "example"+str((int(list(objs["examples"].keys())[-1].split("example")[-1])+1))
objs["examples"][last_example_idx] = dict_base


if write:
    print(objs["examples"][last_example_idx])
    with open(examples_path, 'w') as json_file:
        json.dump(objs, json_file,indent=2)


# PreCode

In [ ]:
import json
train = json.load(open('../data/dataset/train_data.json','r'))

In [ ]:
expl30 = train['examples']['example4']['udp']

plt.figure(figsize=(16,8))
markers = ['*', 'o', '^', 's']
for marker, (k, v) in zip(markers, expl30.items()):
    udp = [vv for _, vv in v.items()]
    plt.plot(udp,'{}-'.format(marker),label=k, markersize=12)

plt.legend(fontsize=20)
plt.xlabel(r'$time \, (s)$', fontsize=20)
plt.ylabel(r"$ UDP \, Throughput \, (Mbps)$", fontsize=20)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)

save_obj = ['udp-thr-metric']
plt.savefig('DataFigures/2ornn/{}.eps'.format(*save_obj), format='eps')